In [1]:
import sys
import json
import csv
import random
import time
import torch
import torch.nn as nn
import numpy as np
import fastText
from utils import *
import torch.nn.functional as F


In [105]:
use_en = np.load('data/snips_processed/USE-en.npy')
use_en = np.squeeze(use_en, axis=1)
labs = np.load('data/snips_processed/labs.npy')


dataset, lab2id, id2lab =import_data('data/snips_processed/snips.csv')
sents = dataset[:,2]

In [3]:
#sv_model = fastText.load_model('data/cc.sv.300.bin')


In [78]:
def sentence_m(sentence, model, max_len):
    result = []
    words = ['sossos'] + sentence.split() + ['eoseos']
    
    for word in words:
        result.append(model.get_word_vector(word))
    

    if len(words) < max_len:
        result.extend([np.zeros(300)] * (max_len  -  len(words)))
    return np.array(result)
    
    
def prepare_sentences(sents, preprocess=False):       
    sents = pre_process_text(sents)  if preprocess else sents
    lens = list(map(lambda x: x.count(' ') + 3, sents))
    max_len = max(lens)
    m = np.stack(list(map(lambda x:sentence_m(x, sv_model, max_len), sents)))
    return m, lens

In [84]:
data, data_lens = prepare_sentences(sents, True)

35

### Код Архитектуры

In [6]:
sys.exit()

filter_sizes = [2,3,5]
num_filters = 512
drop = 0.3

inputs = Input(shape=(MAX_LEN,), dtype='int32')
embedding = Embedding(input_dim=len(word_index) + 1, # dict size
                      output_dim=EMB_DIM, 
                      weights=[embedding_matrix],
                      input_length=MAX_LEN, # number of words at one batch
                      trainable=False)(inputs)

reshape = Reshape((MAX_LEN,EMB_DIM,1))(embedding)  #words*embedding matrix

# 512 filters that takes 2-5 words with all embedding numbers and output single value.
# 2d -> 1d because emb dim in kernel size + dimention on number  of filters. Single filter outputs MAX_LEN - filter_sizes[0] + 1 values
conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], EMB_DIM), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], EMB_DIM), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], EMB_DIM), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

#gets single value for each filter. 512 values total for each pool. Why strides?
maxpool_0 = MaxPool2D(pool_size=(MAX_LEN - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(MAX_LEN - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(MAX_LEN - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

lstm_1 = Bidirectional(LSTM(256, dropout=0.5, recurrent_dropout=0.5))(embedding) # it was 300 units
flatten_lstm = Reshape((1, 1, 512))(lstm_1)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, 
                                           flatten_lstm])
flatten = Flatten()(concatenated_tensor)



SystemExit: 

In [35]:
class ConvNet(nn.Module):
    def __init__(self, max_s_len , emb_dim = 300 , out_size = 7):
        super(ConvNet, self).__init__()
        
        #(batch, sent_len, emb_dim)
        
        self.emb_dim = emb_dim
        self.out_size =out_size
        self.max_s_len = max_s_len
        self.kernel_sizes = [2,3,5]

        self.max_pool_kernel_size = [(self.max_s_len - x + 1, 1) for x in self.kernel_sizes]

        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=128, kernel_size=(self.kernel_sizes[0], self.emb_dim), )

        
    def forward(self, x):
        
        # add chanels dimention:
        x = x.unsqueeze(1)
        
        x1 = self.conv1(x)
        x1 = F.max_pool2d(F.relu(x1), self.max_pool_kernel_size[0])
        return x1

In [36]:
cn = ConvNet(19)

t = torch.tensor(a).float()
out  = cn.forward(t)

out.shape

torch.Size([10, 128, 1, 1])

In [26]:
class BiLstmNet(nn.Module):
    def __init__(self, max_s_len , emb_dim = 300 , out_size = 7):
        super(BiLstmNet, self).__init__()
        
        #(batch, sent_len, emb_dim)
        
        self.emb_dim = emb_dim
        self.out_size =out_size
        self.max_s_len = max_s_len
        
        #dont add dropout  to last layer since there is only one layer
        self.lstm = nn.LSTM(input_size = self.emb_dim, hidden_size = 256,
                         batch_first=True, bidirectional=True)
        
        
    def forward(self, x, lens):
        ps = nn.utils.rnn.pack_padded_sequence(x, lens, batch_first=True, enforce_sorted=False)
        
        out, (h,c) = self.lstm(ps)
        
        return out, h

In [30]:
bln =  BiLstmNet(19)
out, h = bln(t, lens)


(torch.Size([109, 512]),
 torch.Size([2, 10, 256]),
 tensor(0.0159, grad_fn=<SelectBackward>),
 tensor(0.0121, grad_fn=<SelectBackward>))

In [212]:
class RNN(nn.Module):
    def __init__(self, max_s_len , emb_dim = 300, out_size = 512):
        super(RNN, self).__init__()
        
        #(batch, sent_len, emb_dim)
        
        self.emb_dim = emb_dim
        self.out_size =out_size
        self.max_s_len = max_s_len
        self.kernel_sizes = [2,3,5]
        self.cnn_chan  = 128
        self.lstm_hid  = 256
        
        self.drop = nn.Dropout(0.35)

        self.max_pool_kernel_size = [(self.max_s_len - x + 1, 1) for x in self.kernel_sizes]

        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=self.cnn_chan, kernel_size=(self.kernel_sizes[0], self.emb_dim), )
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=self.cnn_chan, kernel_size=(self.kernel_sizes[1], self.emb_dim), )
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=self.cnn_chan, kernel_size=(self.kernel_sizes[2], self.emb_dim), )
        
        #dont add dropout  to last layer since there is only one layer
        self.lstm = nn.GRU(input_size = self.emb_dim, hidden_size = self.lstm_hid,
                         batch_first=True, bidirectional=True)
    
    
        self.lin = nn.Linear(self.lstm_hid*2 + self.cnn_chan * 3, out_size)
        
        
        
    def forward(self, x, lens):
        
        # add chanels dimention:
        xc = x.unsqueeze(1)
        
        x1 = self.conv1(xc)
        x1 = F.max_pool2d(F.relu(x1), self.max_pool_kernel_size[0])
        x1 = x1.squeeze(3).squeeze(2)
        
        x2 = self.conv1(xc)
        x2 = F.max_pool2d(F.relu(x2), self.max_pool_kernel_size[1])
        x2 = x2.squeeze(3).squeeze(2)

        x3 = self.conv1(xc)
        x3 = F.max_pool2d(F.relu(x3), self.max_pool_kernel_size[2])
        x3 = x3.squeeze(3).squeeze(2)

        ps = nn.utils.rnn.pack_padded_sequence(x, lens, batch_first=True, enforce_sorted=False)
        
        _, (h) = self.lstm(ps)
        lstm_out = torch.cat((h[1],h[0]), dim =1)

        out = torch.cat((x1,x2,x3, lstm_out), dim=1)
        
        out = self.drop(out)
        out = self.lin(out)
        
        return out

In [213]:
a, lens = prepare_sentences(sents[:10], True)

net = RNN(19)
out = net(t, lens)
a.shape, out.shape, len(use_en)

((10, 19, 300), torch.Size([10, 512]), 13784)

In [199]:
def closest_index(request, dots, forbiden_index=-1):
    dists = np.linalg.norm(dots-request, axis=1)
    res =  np.argmin(dists)
    
    if res == forbiden_index:
        dists[res] = np.inf
        return np.argmin(dists)
    return res


def train(model, data_in, lens, expected_out, criterion, optimizer):
    model.train()
    model.zero_grad()
    loss = 0

    # vectors = torch.tensor(vectors).float()
    # labels = torch.tensor(labels)

    model_out = model.forward(data_in, lens)
    loss += criterion(model_out, expected_out)

    loss.backward()
    optimizer.step()

    return loss.item() / len(expected_out)
    
def evaluate(model, data_in, lens, labs, expected_out, criterion):
    model.eval()
    with torch.no_grad():
        
        model_out = model.forward(data_in, lens)
        right = 0
        
        for i  in range(len(model_out)):
            predicted_use = model_out[i]
            true_lab = labs[i]
        
            predicted_index = closest_index(predicted_use, expected_out, forbiden_index=i)
            predicted_lab = labs[predicted_index]
            
            if predicted_lab == true_lab:
                right += 1
            
        loss = criterion(model_out, expected_out)
        return loss.item(), right/len(model_out)

In [207]:
def full_train_model(train_data, train_out, test_data, test_out, lens, test_lens, test_labs, verbose = False, runs = 20, net  = None):
    max_s_len = train_data.shape[1]
    if net is None:
        net = RNN(max_s_len = max_s_len)
    optimizer = torch.optim.Adam(net.parameters())
    criterion = torch.nn.MSELoss()
    
    
    data_in = torch.tensor(train_data).float()
    data_in_t = torch.tensor(test_data).float()
    ex_out = torch.tensor(train_out).float()
    ex_out_t = torch.tensor(test_out).float()
        
    print('matrices ready, starting training')
    t = time.time()
    for i in range(runs):
        loss = train(net, data_in, lens, ex_out, criterion, optimizer,)
        if verbose and not i% 50:
            #print('#{:3d}, {:5d} sec.'.format(i, int(time.time()-t)))
            eval_loss, acc = evaluate(net, data_in_t, test_lens, test_labs, ex_out_t, criterion)
            print('#{:3d}, {:5d} sec. train loss: {:.7f}, eval loss: {:.4f}, acc = {:.4f}'.format(i, int(time.time() - t), loss, eval_loss, acc))
    
    eval_loss, acc = evaluate(net, data_in_t, test_lens, test_labs, ex_out_t, criterion)
    return net, eval_loss, acc

In [201]:
train_data = data[:10000]
test_data = data[10000:]

train_out = use_en[:10000]
test_out = use_en[10000:]

train_lens = data_lens[:10000]
test_lens = data_lens[10000:]

test_labs = labs[10000:]

trained_net, eval_loss, acc = \
    full_train_model(train_data,
                     train_out,
                     test_data,
                     test_out,
                     train_lens,
                     test_lens,
                     test_labs,
                     verbose=True,
                     runs = 401)

matrices ready, starting training
#  0,    59 sec. train loss: 0.0000004, eval loss: 0.0021, acc = 0.1874
# 50,  1674 sec. train loss: 0.0000001, eval loss: 0.0010, acc = 0.8380
#100,  2878 sec. train loss: 0.0000001, eval loss: 0.0009, acc = 0.9033
#150,  4035 sec. train loss: 0.0000001, eval loss: 0.0009, acc = 0.9072
#200,  5188 sec. train loss: 0.0000001, eval loss: 0.0008, acc = 0.9152
#250,  6332 sec. train loss: 0.0000001, eval loss: 0.0008, acc = 0.9220
#300,  7474 sec. train loss: 0.0000001, eval loss: 0.0008, acc = 0.9289
#350,  8630 sec. train loss: 0.0000001, eval loss: 0.0007, acc = 0.9284
#400,  9785 sec. train loss: 0.0000001, eval loss: 0.0007, acc = 0.9331


In [219]:
evaluate(trained_net_2,
         torch.tensor(test_data).float(),
         test_lens,
         test_labs,
         torch.tensor(test_out).float(),
         torch.nn.MSELoss()) 

(0.0006925008492544293, 0.9371035940803383)

In [203]:
eval_loss, acc

((0.000723589735571295, 0.9331395348837209), 0.9331395348837209)

In [210]:
torch.save(trained_net_2, 'models/RNN-c128t3-lstm256-500')


/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [163]:
use_model = torch.load('models/USE-linear')
use_model.eval()

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.activation.LogSoftmax' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Baseline(
  (W): Linear(in_features=512, out_features=7, bias=True)
  (out): LogSoftmax()
)

In [218]:
predicted_use = gru500.forward(torch.tensor(test_data).float(), test_lens)
predicted_labs_dist = use_model.forward(predicted_use.unsqueeze(1))
right = 0
for i  in range(len(predicted_labs_dist)):
        k, v = predicted_labs_dist[i].topk(1)
        predicted, true = v.item(), test_labs[i].item()
        if predicted == true:
            right +=1
right/len(predicted_labs_dist)

0.9624735729386892

In [214]:
gru300 = torch.load('models/RNN-c128t3-gru256-03')
gru300.eval()

RNN(
  (drop): Dropout(p=0.35)
  (conv1): Conv2d(1, 128, kernel_size=(2, 300), stride=(1, 1))
  (conv2): Conv2d(1, 128, kernel_size=(3, 300), stride=(1, 1))
  (conv3): Conv2d(1, 128, kernel_size=(5, 300), stride=(1, 1))
  (lstm): GRU(300, 256, batch_first=True, bidirectional=True)
  (lin): Linear(in_features=896, out_features=512, bias=True)
)

In [216]:
gru500, eval_loss, acc = \
    full_train_model(train_data,
                     train_out,
                     test_data,
                     test_out,
                     train_lens,
                     test_lens,
                     test_labs,
                     verbose=True,
                     runs = 201, net = gru300)

matrices ready, starting training
#  0,    46 sec. train loss: 0.0000001, eval loss: 0.0008, acc = 0.9101
# 50,  1191 sec. train loss: 0.0000001, eval loss: 0.0007, acc = 0.9236
#100,  2325 sec. train loss: 0.0000001, eval loss: 0.0007, acc = 0.9345
#150,  3425 sec. train loss: 0.0000001, eval loss: 0.0007, acc = 0.9374
#200,  4533 sec. train loss: 0.0000001, eval loss: 0.0007, acc = 0.9419


In [217]:
eval_loss

0.0006778779206797481